In [1]:
import time
from pprint import pprint
import numpy as np
##
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
##
import gladier.tests
from gladier.client import GladierClient as GladierBaseClient

In [18]:
# fxc = FuncXClient(force_login=True)

Please paste the following URL in a browser:
https://auth.globus.org/v2/oauth2/authorize?client_id=4cf29807-cf21-49ec-9443-ff9a3fb9f81c&redirect_uri=https%3A%2F%2Fauth.globus.org%2Fv2%2Fweb%2Fauth-code&scope=https%3A%2F%2Fauth.globus.org%2Fscopes%2Ffacd7ccc-c5f4-42aa-916b-a0e270e2c2a9%2Fall+urn%3Aglobus%3Aauth%3Ascope%3Asearch.api.globus.org%3Aall+openid&state=_default&response_type=code&code_challenge=N1ancoR09Wokv8elFhzhHBVpokLl6ehtAWGic6Rysac&code_challenge_method=S256&access_type=offline&prefill_named_grant=FuncX+SDK+Login
Please Paste your Auth Code Below: 


In [2]:
class KanzusTriggers:
    def __init__(self, folder_path):
        self.observer = Observer()
        self.folder_path = folder_path

    def run(self):
        print("Kanzus Triggers Started")

        event_handler = Handler()
        self.observer.schedule(event_handler, self.folder_path, recursive = True)
        self.observer.start()
        try:
            while True:
                time.sleep(1)
        except:
            self.observer.stop()
            print("Kanzus Triggers Stopped")
  
        self.observer.join()

class Handler(FileSystemEventHandler):
    @staticmethod
    def on_any_event(event):
        if event.is_directory:
            return None
        elif event.event_type == 'created':

            #event.src_path is the file watchdog found
            # Event is created, you can process it now
            KanzusLogic(event.src_path,f_pattern=None,f_ext=None, n_batch=256)
  

def KanzusLogic(cbf_file,f_pattern=None,f_ext=None, n_batch=256):
    if not f_pattern:
        f_pattern = r'(\w+)_(\d+)_\d+.cbf'
    cbf_parse = re.compile(f_pattern)
    cbf_parse(cbf_file)
    print(cbf_file, cbf_parse)

        ##cbf is created
        ##n_batch runs plot
        ##cbf name  %n_batch==0
        ## 


# exp = KanzusTriggers('.')
# exp.run()

In [3]:
from gladier_kanzus.flows.search_flow import flow_definition

class KanzusXYSearchClient(GladierBaseClient):
    client_id = 'e6c75d97-532a-4c88-b031-8584a319fa3e'

    gladier_tools = [
        'gladier_kanzus.tools.XYSearch',
        'gladier_kanzus.tools.CreatePhil',
        'gladier_kanzus.tools.DialsStills',
    ]
    flow_definition = flow_definition


search_client = KanzusXYSearchClient()

[DEBUG] gladier.client::__init__() No authorizers provided, loading from disk.


In [4]:
from funcx.sdk.client import FuncXClient
fxc = FuncXClient()
from gladier_kanzus.tools.dials_stills import funcx_stills_process as stills_cont
cont_dir =  '/home/rvescovi/.funcx/containers/'
container_name = "dials_v1.simg"
dials_cont_id = fxc.register_container(location=cont_dir+'/'+container_name,container_type='singularity')
stills_cont_fxid = fxc.register_function(stills_cont, container_uuid=dials_cont_id)

In [11]:
##theta
conf = {'local_endpoint': '8f2f2eab-90d2-45ba-a771-b96e6d530cad',
        'queue_endpoint': '23519765-ef2e-4df2-b125-e99de9154611',
        }
# conf = {'local_endpoint': '8f2f2eab-90d2-45ba-a771-b96e6d530cad',
        # 'queue_endpoint': '8f2f2eab-90d2-45ba-a771-b96e6d530cad',
        # }
##cooley
# conf = {'local_endpoint': '83e95e2e-fd70-45ea-9467-5efe5d95ff11',
#         'queue_endpoint': 'd26622fb-3bef-44df-8874-fcfdfbcc29fd',
#         }

data_dir = '/eagle/APSDataAnalysis/SSX/Demo/test'
proc_dir = f'{data_dir}/xy'

flow_input = {
    "input": {
        #Processing variables
        "proc_dir": proc_dir,
        "data_dir": data_dir,

        #Dials specific variables.
        "input_files": "Ohakune_33_{00000..00010}.cbf", 
        "input_range": "00000..00010",
        "nproc": 10,
        "beamx": "-214.400",
        "beamy": "218.200",
        "step": "0.3",

        # funcX endpoints
        "funcx_local_ep": conf['local_endpoint'],
        "funcx_queue_ep": conf['queue_endpoint'],

        # container hack for stills
        "stills_cont_fxid": stills_cont_fxid
    }
}


In [14]:
pprint(flow_input['input'])

{'beamx': '-214.400',
 'beamy': '218.200',
 'data_dir': '/eagle/APSDataAnalysis/SSX/Demo/test',
 'funcx_local_ep': '8f2f2eab-90d2-45ba-a771-b96e6d530cad',
 'funcx_queue_ep': '23519765-ef2e-4df2-b125-e99de9154611',
 'input_files': 'Ohakune_33_{00000..00010}.cbf',
 'input_range': '00000..00010',
 'nproc': 10,
 'proc_dir': '/eagle/APSDataAnalysis/SSX/Demo/test/xy',
 'step': '0.3',
 'stills_cont_fxid': '12c535f2-65dc-4e28-929a-3ecfc80d7f30'}


In [16]:
phils_flow = search_client.start_flow(flow_input=flow_input)

[DEBUG] gladier.client::get_funcx_function_ids() Checking functions for <gladier_kanzus.tools.XYSearch object at 0x7f4145046970>
[DEBUG] gladier.client::get_funcx_function_ids() Checking functions for <gladier_kanzus.tools.CreatePhil object at 0x7f4145046a60>
[DEBUG] gladier.client::get_funcx_function_ids() Checking functions for <gladier_kanzus.tools.DialsStills object at 0x7f4145046850>
[INFO] gladier.client::start_flow() Started flow kanzus_xy_search_client flow id "e79bb595-4efc-4389-872c-12469ad27262" with action "5b7182ea-f812-4fcf-b355-93f45e3e8785"
